In [4]:
import numpy as np
from numpy import random as rd
import matplotlib.pylab as plt
import pyprind

In [101]:
############### changable parameter ###############
n = 50                                 #repeat times
t_exp = 60                              #exposure time(sec)
times = 30                              #number of transmissions
max_amp = 100                           #amplitude at lowest frequency
pattern = 'primary'                     #transit pattern ('primary' or 'secondary')

In [102]:
############### shortcut name ###############
fname = '/Users/shohei/Desktop/Shohei/Fresnel/Detector/Paper/Data'

In [103]:
############### constant parameter ###############
h = 6.626070040*10**(-27)                               # Plank constant(erg*s)
c = 2.99792458*10**10                                   # speed of light(cm/s)
k = 1.38064852*10**(-16)                                # Boltzmann constant(erg/K)
T_star = np.load('{}/temperature.npy'.format(fname))    # star temperature(K)
T_planet = 288.2                                        # planet temperature(K)
R_Sun = 6.960*10**10                                    # Solar radius(cm)
R_Earth = 6.378137*10**8                                # Earth radius(cm)
R = np.load('{}/radius.npy'.format(fname))              # star radius(R_Sun)
R_star = R*R_Sun                                        # star radius(cm)
R_planet = R_Earth                                      # planet radius(cm)
p = R_planet/R_star                                     # radius ratio
a = np.load('{}/semi.npy'.format(fname))                # semi major axis(cm)
P = np.load('{}/period.npy'.format(fname))              # period(sec)
P_tra = P*R_star/np.pi/a                                # transit period(sec)
t_one = (P_tra*3/t_exp).astype(np.int64)                # the number of plots in one transit observation
t_total = t_one*times                                   # total times(2min)
t0 = [np.linspace(1, t_one[TT], t_one[TT])
      for TT in range(len(T_star))]                     # time for one transmission(min)
t = [np.linspace(1, t_total[TT], t_total[TT])
     for TT in range(len(T_star))]                      # time for total(min)
lamb = np.load('{}/lamb.npy'.format(fname))             # wavelength(µm)
R1 = 100                                                # resolution of detector1(5~10μm)
R2 = 100                                                # resolution of detector2(10~20μm)
dl1 = (5+10)/2/R1                                       # wavelength band(6~11µm)
dl2 = (10+20)/2/R2                                      # wavelength band(11~18µm)
sci_pix_1 = 200*int(594*dl1/(10-5))                     # the number of science pixels by wavelength on detector1
sci_pix_2 = 200*int(594*dl2/(20-10))                    # the number of science pixels by wavelength on detector2
back_pix_1 = sci_pix_1                                  # the number of background pixels by wavelength on detector1
back_pix_2 = sci_pix_2                                  # the number of background pixels by wavelength on detector2
ref_pix = 1024*1024-200*594*2                           # the number of reference pixels on detector1,2
dark_value = 0.2*t_exp                                  # dark current(electron)
offset_value = 14*np.sqrt(8*3/t_exp)                    # offset value(electron)
n_gate = 4                                              # the number of gates
zodi = np.load('{}/zodi.npy'.format(fname))             # zodiacal light by wavelength(electron)
#zodi = np.array([0]*len(lamb))
#dark_value = 0
#offset_value = 0

In [104]:
############### make the gain fluctuation ###############
def gain_fluctuation(TT):
    period = np.linspace(60*2, t_one[TT]*60*4, 10000)
    frequency = 1/period
    for tt in range(times):
        fluctuation = np.sum([Fluctuation(f, t0[TT]*60) for f in frequency], axis=0)*max_amp*10**(-6)*np.sqrt(frequency[-1])
        gain_one = (0.9+rd.random()/10)*(1+fluctuation)
        if tt==0:
            gain = gain_one
        else:
            gain = np.hstack((gain, gain_one))
    return gain

def Fluctuation(f, t):
    return np.sin(2*np.pi*(f*t+rd.random()))/np.sqrt(f)

In [105]:
############### make the science curves ###############
def science(TT, ll, gain, n_pix):
    tra = tra_one[ll]
    for tt in range(times-1):
        tra = np.hstack((tra, tra_one[ll]))
    x_star = (h*c)/(lamb[ll]*10**(-4)*k*T_star[TT])
    x_zodi = (h*c)/(lamb[ll]*10**(-4)*k*275)
    bose_star = abs(np.exp(x_star)/(np.exp(x_star)-1))
    bose_zodi = abs(np.exp(x_zodi)/(np.exp(x_zodi)-1))
    D_sci_ijk = np.array([rd.normal((tra[kk]+zodi[ll])/n_pix+dark_value,
                                   np.sqrt(((bose_star*tra[kk]+bose_zodi*zodi[ll])/n_pix+dark_value+offset_value)/(n_pix/n_gate)))
                          for kk in range(t_total[TT])], float)*gain
#    if ll==66:
#        S = D_sci_ijk[0:t_one[TT]]
#        tra_top = [S[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]]
#        sig_top = np.std(tra_top)/np.sqrt(len(tra_top))
#        tra_bottom = [S[kk] for kk in range(t_one[TT]) if z[kk]<1-p[TT]]
#        sig_bottom = np.std(tra_bottom)/np.sqrt(len(tra_bottom))
#        sig = np.sqrt(sig_top**2+sig_bottom**2)
#        NS = sig/S.mean()*10**6/2/np.sqrt(30)
        #print('1/SN=', NS)
    return D_sci_ijk

In [106]:
############### make the background curves ###############
def background(TT, ll, gain, n_pix):
    x_zodi = (h*c)/(lamb[ll]*10**(-4)*k*275)
    bose_zodi = abs(np.exp(x_zodi)/(np.exp(x_zodi)-1))
    D_back_ijk = rd.normal(zodi[ll]/n_pix+dark_value,
                           np.sqrt(((bose_zodi*zodi[ll])/n_pix+dark_value+offset_value)/(n_pix/n_gate)), t_total[TT])*gain
    return D_back_ijk

In [107]:
############### make the reference curves ###############
def reference(TT, gain, n_pix):
    D_ref_ijk = rd.normal(dark_value, np.sqrt((dark_value+offset_value)/(n_pix/n_gate)), t_total[TT])*gain
    return D_ref_ijk

In [108]:
############### each average curve ###############
def unify(nn, TT):
    D_sci_ave_k = np.array([[[0]*t_total[TT]]*len(lamb)]*n_gate, float)
    D_back_ave_k = np.array([[[0]*t_total[TT]]*len(lamb)]*n_gate, float)
    D_ref_ave_k = np.array([[[0]*t_total[TT]]*n_gate]*2, float)
    for jj in range(n_gate):
        #gain = 1.0
        gain_1 = gain_fluctuation(TT)
        gain_2 = gain_fluctuation(TT)
        for ll in range(len(lamb)):
            if lamb[ll]<10:
                sci_pix = sci_pix_1
                back_pix = back_pix_1
                gain = gain_1
            else:
                sci_pix = sci_pix_2
                back_pix = back_pix_2
                gain = gain_2
            D_sci_ave_k[jj, ll] = science(TT, ll, gain, sci_pix)
            D_back_ave_k[jj, ll] = background(TT, ll, gain, back_pix)
        #D_ref_ave_k[0, jj] = reference(TT, gain, ref_pix)
        #D_ref_ave_k[1, jj] = reference(TT, gain, ref_pix)
        D_ref_ave_k[0, jj] = reference(TT, gain_1, ref_pix)
        D_ref_ave_k[1, jj] = reference(TT, gain_2, ref_pix)
    D_sci_ave = np.mean(D_sci_ave_k, axis=0)
    D_back_ave = np.mean(D_back_ave_k, axis=0)
    D_ref_ave = np.mean(D_ref_ave_k, axis=1)
    return D_sci_ave, D_back_ave, D_ref_ave

In [109]:
############### calibrate the gain fluctuation ###############
def gain_calibration(nn, TT):
    D_sci_ave, D_back_ave, D_ref_ave = unify(nn, TT)
    D_sci = np.array([[0]*t_total[TT]]*len(lamb), float)
    D_sub = np.array([[0]*t_total[TT]]*len(lamb), float)
    for ll in range(len(lamb)):
        if lamb[ll]<10:
            D_ref = D_ref_ave[0]
        else:
            D_ref = D_ref_ave[1]
        for tt in range(times):
            sci_mean = 0
            back_mean = 0
            ref_mean = 0
            for kk in range(tt*t_one[TT], (tt+1)*t_one[TT]):
                sci_mean += D_sci_ave[ll, kk]
                back_mean += D_back_ave[ll, kk]
                ref_mean += D_ref[kk]
            sci_mean /= t_one[TT]
            back_mean /= t_one[TT]
            ref_mean /= t_one[TT]
            for kk in range(tt*t_one[TT], (tt+1)*t_one[TT]):
                D_sci[ll, kk] = D_sci_ave[ll, kk]-back_mean
                D_sub[ll, kk] = D_sci_ave[ll, kk]-(D_ref[kk]-ref_mean)*sci_mean/ref_mean-back_mean
    return D_sci, D_sub

In [110]:
############### evaluate this simulation ###############
def evaluation(nn, TT):
    global tra_one, z
    tra_one = np.load('{}/{}_{}.npy'.format(fname, pattern, int(T_star[TT])))
    z = np.load('{}/parameter_{}.npy'.format(fname, int(T_star[TT])))
    D_sci, D_sub = gain_calibration(nn, TT)
    for ll in range(len(lamb)):
        D_sci_bin = np.mean([[D_sci[ll, kk+tt*t_one[TT]] for kk in range(t_one[TT])] for tt in range(times)], axis=0)
        D_sci_bin_norm = D_sci_bin/np.mean([D_sci_bin[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]], axis=0)
        D_sub_bin = np.mean([[D_sub[ll, kk+tt*t_one[TT]] for kk in range(t_one[TT])] for tt in range(times)], axis=0)
        D_sub_bin_norm = D_sub_bin/np.mean([D_sub_bin[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]], axis=0)
        D_mod = tra_one[ll]
        D_sci_flat = average(TT, D_sci[ll])
        D_sub_flat = average(TT, D_sub[ll])
        D_mod_norm = D_mod/D_mod[10]
        D_sci_flat_norm = D_sci_flat/D_sci_flat[10]
        D_sub_flat_norm = D_sub_flat/D_sub_flat[10]
        sci_eva[nn, ll] = (D_mod_norm[int(t_one[TT]/2)]-D_sci_flat_norm[int(t_one[TT]/2)])*10**6
        sub_eva[nn, ll] = (D_mod_norm[int(t_one[TT]/2)]-D_sub_flat_norm[int(t_one[TT]/2)])*10**6
        sigma_top_sci = [D_sci_bin_norm[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]]
        sigma_bottom_sci = [D_sci_bin_norm[kk] for kk in range(t_one[TT]) if z[kk]<1-p[TT]]
        sigma_top_sub = [D_sub_bin_norm[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]]
        sigma_bottom_sub = [D_sub_bin_norm[kk] for kk in range(t_one[TT]) if z[kk]<1-p[TT]]
        sci_sig[nn, ll] = np.sqrt(np.std(sigma_top_sci)**2/len(sigma_top_sci)+np.std(sigma_bottom_sci)**2/len(sigma_bottom_sci))*10**6
        sub_sig[nn, ll] = np.sqrt(np.std(sigma_top_sub)**2/len(sigma_top_sub)+np.std(sigma_bottom_sub)**2/len(sigma_bottom_sub))*10**6
#        if nn==0 and ll==66:
#            plt.plot(D_sci_bin_norm, 'ro', c='r', markersize='3')
#            plt.plot(D_mod_norm, c='g')
#            plt.show()
#            tra_top = [D_sci_bin_norm[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]]
            #tra_top = [D_sci[ll, kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]]
            #tra_top = [D_sci[ll, kk+tt*t_one[TT]] for tt in range(times) for kk in range(t_one[TT]) if z[kk]>1+p[TT]]
#            sig_top = np.std(tra_top)/np.sqrt(len(tra_top))
#            tra_bottom = [D_sci_bin_norm[kk] for kk in range(t_one[TT]) if z[kk]<1-p[TT]]
            #tra_bottom = [D_sci[ll, kk] for kk in range(t_one[TT]) if z[kk]<1-p[TT]]
            #tra_bottom = [D_sci[ll, kk+tt*t_one[TT]] for tt in range(times) for kk in range(t_one[TT]) if z[kk]<1-p[TT]]
#            sig_bottom = np.std(tra_bottom)/np.sqrt(len(tra_bottom))
#            sig = np.sqrt(sig_top**2+sig_bottom**2)
#            NS = sig/D_sci_bin_norm.mean()*10**6
            #print('1/SN=', NS)
    pbar.update()

In [111]:
############### option ###############
def average(TT, X):
    if len(X)==t_one[TT]:
        average = X
    else:
        average = np.mean([[X[kk+tt*t_one[TT]] for kk in range(t_one[TT])] for tt in range(times)], axis=0)
    top = sum([average[kk] for kk in range(t_one[TT]) if z[kk]>1+p[TT]])
    n_top = sum([1 for kk in range(t_one[TT]) if z[kk]>1+p[TT]])
    bottom = sum([average[kk] for kk in range(t_one[TT]) if z[kk]<1-p[TT]])
    n_bottom = sum([1 for kk in range(t_one[TT]) if z[kk]<1-p[TT]])
    for kk in range(t_one[TT]):
        if z[kk]>1+p[TT]:
            average[kk] = top/n_top
        elif z[kk]<1-p[TT]:
            average[kk] = bottom/n_bottom
    return average

In [112]:
pbar = pyprind.ProgBar(n)
sci_eva = np.array([[0]*len(lamb)]*n, float)
sub_eva = np.array([[0]*len(lamb)]*n, float)
sci_sig = np.array([[0]*len(lamb)]*n, float)
sub_sig = np.array([[0]*len(lamb)]*n, float)
[evaluation(nn, 3) for nn in range(n)]
print('Success')

0% [##############################] 100% | ETA: 00:00:00

Success



Total time elapsed: 03:51:15


In [113]:
print('sci_mean:', sci_eva[:, 66].mean())
print(' sci_std:', sci_eva[:, 66].std())
print('sub_mean:', sub_eva[:, 66].mean())
print(' sub_std:', sub_eva[:, 66].std())

sci_mean: 0.9624757997106493
 sci_std: 31.414157842418362
sub_mean: -0.8572143797458409
 sub_std: 6.3329364328634155


In [13]:
bose = 2.2831700551572647     # bose factor at 10µm
40/25

1.6

1.6

In [277]:
S_mean = 1700000
S = rd.normal(S_mean, np.sqrt(S_mean*bose), 1000)
N = S.std()/np.sqrt(1000)
N/S.mean()*10**6

36.662066869767756

In [59]:
t1_100  = 15.2364729884153      # times=1, n=100
t2_100  = 10.964809101511907    # times=2, n=100
t10_100 = 4.007039661850969     # times=10, n=100
t20_100 = 3.205685848661847     # times=20, n=100
t30_30  = 3.214860354063453     # times=30, n=30
t30_50  = 2.8986181423204576    # times=30, n=50
t30_100 = 2.663247199720826     # times=30, n=100

In [100]:
v30_100 = 6.322063587329086    # times=30, max_amp=100
v30_50 = 6.666680328914093      # times=30, max_amp=50

In [53]:
(t20_100/t30_100)**2

1.448835110561374

In [43]:
def binning(data, R, n):
    i = int(100/R)
    new_data = np.array([[0]*(int(len(lamb)/i)+1)]*n, float)
    for ll in range(int(len(lamb)/i)+1):
        m = 0
        for jj in range(i):
            if len(lamb)<=ll*i+jj:
                break
            new_data[:, ll] += data[:, ll*i+jj]
            m += 1
        new_data[:, ll] /= m
    return new_data

In [60]:
sci_eva_20 = binning(sci_eva, 20, 50)

In [62]:
print(sci_eva[:, 0].std())
print(sci_eva_20[:, 0].std())
print(sci_eva[:, 0].std()/sci_eva_20[:, 0].std())

0.8182169613085248
0.3982366840929693
2.0545996739906314
